<a href="https://colab.research.google.com/github/pervez72/1-Google-colab/blob/Project/liniear_regrassion_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import kagglehub
import pandas as pd
import numpy as np
# Download latest version
path = kagglehub.dataset_download("mirichoi0218/insurance")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'insurance' dataset.
Path to dataset files: /kaggle/input/insurance


In [15]:
df=pd.read_csv("/kaggle/input/insurance/insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


In [21]:

# Split dataset before encoding
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [22]:
# Encode cetagorical variable
label_encoder = {}
for col in ['sex', 'smoker', 'region']:
  le = LabelEncoder()
  train_df[col] = le.fit_transform(train_df[col])
  test_df[col] = le.transform(test_df[col])
  label_encoder[col] = le


In [23]:
# Features and target
X_train = train_df.drop(columns=['charges'])
y_train = train_df['charges']

X_test = test_df.drop(columns=['charges'])
y_test = test_df['charges']

In [24]:
print(X_train.head())
print(y_train.head())

      age  sex    bmi  children  smoker  region
560    46    0  19.95         2       0       1
1285   47    0  24.32         0       0       0
1142   52    0  24.86         0       0       2
969    39    0  34.32         5       0       2
486    54    0  21.47         3       0       1
560      9193.83850
1285     8534.67180
1142    27117.99378
969      8596.82780
486     12475.35130
Name: charges, dtype: float64


In [25]:
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
print(X_train)

[[ 0.47222651 -1.0246016  -1.75652513  0.73433626 -0.50874702 -0.45611589]
 [ 0.54331294 -1.0246016  -1.03308239 -0.91119211 -0.50874702 -1.35325561]
 [ 0.8987451  -1.0246016  -0.94368672 -0.91119211 -0.50874702  0.44102382]
 ...
 [ 1.3252637   0.97598911 -0.89153925 -0.91119211 -0.50874702 -1.35325561]
 [-0.16755139 -1.0246016   2.82086429  0.73433626  1.96561348  1.33816354]
 [ 1.1120044   0.97598911 -0.10932713 -0.91119211 -0.50874702  1.33816354]]


In [27]:
# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [28]:
print(y_train_tensor)
print(y_train_tensor.shape)

tensor([[ 9193.8389],
        [ 8534.6719],
        [27117.9941],
        ...,
        [11931.1250],
        [46113.5117],
        [10214.6357]])
torch.Size([1070, 1])


In [29]:
print(X_test_tensor)
print(X_test_tensor.shape)

tensor([[ 0.4011, -1.0246, -0.8915,  0.7343, -0.5087, -1.3533],
        [-0.2386, -1.0246, -0.0895, -0.9112, -0.5087, -0.4561],
        [ 1.7518, -1.0246, -0.6085, -0.9112,  1.9656, -0.4561],
        ...,
        [-0.0965,  0.9760, -0.4197, -0.0884, -0.5087, -1.3533],
        [ 1.0409, -1.0246,  2.7894, -0.9112,  1.9656,  0.4410],
        [ 0.8277, -1.0246,  0.6025, -0.0884, -0.5087,  1.3382]])
torch.Size([268, 6])


In [31]:
  # Define Neural network model

  class SimpleNNRegressionModel(nn.Module):
    def __init__(self, input_dim):
      super(SimpleNNRegressionModel, self).__init__()
      self.network = nn.Sequential(
          nn.Linear(input_dim, 64),
          nn.ReLU(),
          nn.Linear(64, 128),
          nn.ReLU(),
          nn.Linear(128, 1)
      )

    def forward(self, x):
      return self.network(x)

In [32]:
X_train_tensor.shape

torch.Size([1070, 6])

In [33]:
input_dim = X_train_tensor.shape[1]
model = SimpleNNRegressionModel(input_dim)

In [34]:
print(model)

SimpleNNRegressionModel(
  (network): Sequential(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
  )
)


In [36]:
# Loss and optmiser

criterion = nn.MSELoss()
optimiser = optim.Adam(model.parameters(), lr=0.01)

In [37]:
# Training loop
epochs = 30000

for epoch in range(epochs):
  model.train()
  optimiser.zero_grad()
  predictions = model(X_train_tensor)
  loss = criterion(predictions, y_train_tensor)
  loss.backward()

  optimiser.step()

  if (epoch+1) % 100 == 0:
    print(f"Epoch [{epoch+1}/{epochs}], Loss : {loss.item():.4f}")

Epoch [100/30000], Loss : 45874516.0000
Epoch [200/30000], Loss : 33009442.0000
Epoch [300/30000], Loss : 31251114.0000
Epoch [400/30000], Loss : 28996102.0000
Epoch [500/30000], Loss : 26770578.0000
Epoch [600/30000], Loss : 25338534.0000
Epoch [700/30000], Loss : 24453684.0000
Epoch [800/30000], Loss : 23875558.0000
Epoch [900/30000], Loss : 23483042.0000
Epoch [1000/30000], Loss : 23130226.0000
Epoch [1100/30000], Loss : 22638864.0000
Epoch [1200/30000], Loss : 22051710.0000
Epoch [1300/30000], Loss : 21558554.0000
Epoch [1400/30000], Loss : 21082380.0000
Epoch [1500/30000], Loss : 20643700.0000
Epoch [1600/30000], Loss : 20215608.0000
Epoch [1700/30000], Loss : 19709128.0000
Epoch [1800/30000], Loss : 19328868.0000
Epoch [1900/30000], Loss : 18929972.0000
Epoch [2000/30000], Loss : 18386772.0000
Epoch [2100/30000], Loss : 17704766.0000
Epoch [2200/30000], Loss : 17161462.0000
Epoch [2300/30000], Loss : 16690321.0000
Epoch [2400/30000], Loss : 16196734.0000
Epoch [2500/30000], Loss 

In [39]:
# Model Evaluation

model.eval()
y_pred = model(X_test_tensor).detach().numpy()

In [40]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_test_numpy = y_test_tensor.numpy()

# Calculate metrics
mse = mean_squared_error(y_test_numpy, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test_numpy, y_pred)
r2 = r2_score(y_test_numpy, y_pred)

print(f"MSE : {mse}")
print(f"RMSE : {rmse}")
print(f"MAE : {mae}")
print(f"R2-Score : {r2}")

# 0 --> 0

MSE : 193158928.0
RMSE : 13898.162756278256
MAE : 5448.24169921875
R2-Score : -0.24418997764587402


In [41]:
def predict_charges(age, sex, bmi, children, smoker,region):
  input_data = pd.DataFrame([[age, sex, bmi, children, smoker,region]],
               columns=['age', 'sex', 'bmi', 'children', 'smoker', 'region'])

  for col in ['sex', 'smoker', 'region']:
    input_data[col] = label_encoder[col].transform(input_data[col])
  input_data = scaler.transform(input_data)
  input_tensor = torch.tensor(input_data, dtype=torch.float32)
  predicted_charge = model(input_tensor).item()
  return predicted_charge


In [42]:
predicted = predict_charges(50, 'female', 27.9, 0, 'yes', 'southwest')

In [43]:
print(f"Predicted insurance charge: ${predicted:.2f}")

Predicted insurance charge: $31646.33
